<a href="https://colab.research.google.com/github/philipayazi/Disaster_Tweets/blob/master/Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import pickle
# # obj0, obj1, obj2 are created here...
# # Saving the objects:
# with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump([test_tweet_matrix, train_tweet_matrix, y_train_split, y_test_split, nlp], f)

In [3]:
# Getting back the objects:
with open('objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    test_tweet_matrix, train_tweet_matrix, y_train_split, y_test_split, nlp = pickle.load(f)

## DL Classifier from W2V Features

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [5]:
X_train =train_tweet_matrix
X_test =test_tweet_matrix
y_train =y_train_split
y_test =y_test_split

print('Train X and y', X_train.shape, y_train.shape)
print('Test X and y', X_test.shape, y_test.shape)

Train X and y (5329, 300) (5329,)
Test X and y (2284, 300) (2284,)


In [6]:
# X_scaler = MinMaxScaler().fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

#---------------- No scaling gives better results
X_train_scaled =X_train
X_test_scaled =X_test

In [7]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [8]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


In [9]:
y_train_categorical.shape

(5329, 2)

### Create DL model

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [11]:
# Create model and add layers
# model = Sequential()
# model.add(Dense(units=100, activation='relu', input_dim=20))
# model.add(Dense(units=100, activation='relu'))
# model.add(Dense(units=2, activation='softmax'))

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=300))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))


In [12]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               30100     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 40,402
Trainable params: 40,402
Non-trainable params: 0
_________________________________________________________________


In [14]:
#fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=10,  #60,
    shuffle=True,
    verbose=2,
    validation_split=0.15
)


Train on 4529 samples, validate on 800 samples
Epoch 1/10
4529/4529 - 3s - loss: 0.4870 - accuracy: 0.7699 - val_loss: 0.4836 - val_accuracy: 0.7763
Epoch 2/10
4529/4529 - 0s - loss: 0.4189 - accuracy: 0.8145 - val_loss: 0.4629 - val_accuracy: 0.8000
Epoch 3/10
4529/4529 - 0s - loss: 0.3835 - accuracy: 0.8357 - val_loss: 0.4683 - val_accuracy: 0.7788
Epoch 4/10
4529/4529 - 0s - loss: 0.3564 - accuracy: 0.8459 - val_loss: 0.4557 - val_accuracy: 0.7900
Epoch 5/10
4529/4529 - 0s - loss: 0.3184 - accuracy: 0.8666 - val_loss: 0.4859 - val_accuracy: 0.7987
Epoch 6/10
4529/4529 - 0s - loss: 0.2837 - accuracy: 0.8828 - val_loss: 0.5311 - val_accuracy: 0.7950
Epoch 7/10
4529/4529 - 0s - loss: 0.2497 - accuracy: 0.8995 - val_loss: 0.5345 - val_accuracy: 0.7887
Epoch 8/10
4529/4529 - 0s - loss: 0.2261 - accuracy: 0.9097 - val_loss: 0.5735 - val_accuracy: 0.7912
Epoch 9/10
4529/4529 - 0s - loss: 0.1926 - accuracy: 0.9267 - val_loss: 0.6189 - val_accuracy: 0.7837
Epoch 10/10
4529/4529 - 0s - loss: 

In [15]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=3)
print(
    f"RESULTS - Loss: {model_loss}, Accuracy: {model_accuracy}")

RESULTS - Loss: 0.6522933747064837, Accuracy: 0.8099824786186218


In [16]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [17]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [0 0 0 0 0]
Actual Labels: [1, 0, 1, 0, 0]


## Hyperas ... for hyperparams optimization

In [18]:
# #split into test and train data
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=42)

X_train =X_train_scaled
y_train =y_train_categorical

X_test =X_test_scaled
y_test =y_test_categorical

In [19]:
def data():
    #read in data for analysis 
    
#     all_wells = pd.read_csv('Cleaned_Data/well_cleaned.csv')
#     X1= all_wells[["AVG_CHOKE_SIZE_P","AVG_WHP_P","AVG_WHT_P","BORE_OIL_VOL","BORE_GAS_VOL", "BORE_WAT_VOL"]]
#     y1= all_wells["AVG_DOWNHOLE_PRESSURE"].values.reshape(-1, 1)
#     #split into test and train data
#     X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=42)
    
    with open('objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
        test_tweet_matrix, train_tweet_matrix, y_train_split, y_test_split, nlp = pickle.load(f)
        
    X_train =train_tweet_matrix
    X_test =test_tweet_matrix
    y_train =y_train_split
    y_test =y_test_split
    
    #Create a StandardScater model and fit it to the training data
    X_scaler = StandardScaler().fit(X_train)
    #y_scaler = StandardScaler().fit(y_train)
    
    # # Transform the training and testing data using the X_scaler and y_scaler models
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    y_train_scaled = y_train
    y_test_scaled = y_test
    
    
#     x_train = X_train_scaled.reshape(-1,6)
#     x_test = X_test_scaled.reshape(-1,6)
#     y_train = y_train_scaled.reshape(-1,1)
#     y_test = y_test_scaled.reshape(-1,1)

    x_train = X_train_scaled
    x_test = X_test_scaled
    y_train = y_train_scaled
    y_test = y_test_scaled

    return x_train, y_train, x_test, y_test

In [20]:
from keras import backend
    
################################################
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# mean squared error (mse) for regression  (only for Keras tensors)
def mse(y_true, y_pred):
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

def r_square(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )
#############################################

Using TensorFlow backend.


In [21]:
from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras import backend as K
from keras.utils import print_summary
          
              
def create_model(x_train, y_train, x_test, y_test):
    print(x_train.shape)
    model= Sequential() 
#     model.add(Dense({{choice([50,200,500])}}, input_dim=x_train.shape[1], activation= 'relu'))
#     model.add(Dropout({{uniform(0,1)}}))
#     model.add(Dense({{choice([50,200,500])}},activation= 'relu'))
#     #model.add(Activation('relu'))
#     model.add(Dropout({{uniform(0,1)}}))
#     model.add(Dense(1, activation= 'linear'))
    model.add(Dense( {{choice([50,100,500])}} , activation='relu', input_dim=300))
    model.add(Dense( {{choice([50,100,500])}} , activation='relu'))
    model.add(Dense(units=2, activation='softmax'))
    
    #model.compile(loss='mean_absolute_error', optimizer= 'adam', metrics=[r_square, rmse])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print_summary(model, line_length=None, positions=None, print_fn=None)
    result= model.fit(x_train, y_train,
                      batch_size={{choice([64,128])}},
                      epochs={{choice([10,30,50])}},
                      verbose=2,
                      validation_split =0.15)
    
    score = model.evaluate(x_test, y_test, verbose=0)
    accuracy = score[1]
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

    #validation_acc= np.min(result.history['val_loss'])
    #print('Lowest Validation Loss:', validation_acc)
    #return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model} 

# best_run, best_model= optim.minimize(model=create_model,
#                                      data=data,
#                                      algo=tpe.suggest,
#                                      functions= ['accuracy'], #[rmse,mse,r_square],
#                                      max_evals=5,
#                                      trials=Trials(),
#                                      notebook_name='Disaster_Tweets_hyperasW2VDL')

best_run = optim.minimize(model=create_model,
                          data=data,
                          algo=tpe.suggest,
                          max_evals=10,
                          trials=Trials())


OSError: [Errno 22] Invalid argument: 'C:\\Users\\dviassolo\\RiceBTC\\Disaster_Tweets\\<ipython-input-21-4035455cb4bb>'